In [ ]:
"""This is a minimal example of using Tianshou with MARL to train agents.

Author: Will (https://github.com/WillDudley)

Python version used: 3.8.10

Requirements:
pettingzoo == 1.22.0
git+https://github.com/thu-ml/tianshou
"""

import os
from typing import Optional, Tuple

import gymnasium as gym
import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.utils.net.common import Net

from pettingzoo.classic import tictactoe_v3


def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gym.spaces.Dict)
        else env.observation_space
    )
    if agent_learn is None:
        # model
        net = Net(
            state_shape=observation_space.shape or observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[128, 128, 128, 128],
            device="cuda" if torch.cuda.is_available() else "cpu",
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_learn = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=0.9,
            estimation_step=3,
            target_update_freq=320,
        )

    if agent_opponent is None:
        agent_opponent = RandomPolicy()

    agents = [agent_opponent, agent_learn]
    policy = MultiAgentPolicyManager(agents, env)
    return policy, optim, env.agents


def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""
    return PettingZooEnv(tictactoe_v3.env())


if __name__ == "__main__":
    # ======== Step 1: Environment setup =========
    train_envs = DummyVectorEnv([_get_env for _ in range(10)])
    test_envs = DummyVectorEnv([_get_env for _ in range(10)])

    # seed
    seed = 1
    np.random.seed(seed)
    torch.manual_seed(seed)
    train_envs.seed(seed)
    test_envs.seed(seed)

    # ======== Step 2: Agent setup =========
    policy, optim, agents = _get_agents()

    # ======== Step 3: Collector setup =========
    train_collector = Collector(
        policy,
        train_envs,
        VectorReplayBuffer(20_000, len(train_envs)),
        exploration_noise=True,
    )
    test_collector = Collector(policy, test_envs, exploration_noise=True)
    # policy.set_eps(1)
    train_collector.collect(n_step=64 * 10)  # batch size * training_num

    # ======== Step 4: Callback functions setup =========
    def save_best_fn(policy):
        model_save_path = os.path.join("log", "rps", "dqn", "policy.pth")
        os.makedirs(os.path.join("log", "rps", "dqn"), exist_ok=True)
        torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

    def stop_fn(mean_rewards):
        return mean_rewards >= 0.6

    def train_fn(epoch, env_step):
        policy.policies[agents[1]].set_eps(0.1)

    def test_fn(epoch, env_step):
        policy.policies[agents[1]].set_eps(0.05)

    def reward_metric(rews):
        return rews[:, 1]

    # ======== Step 5: Run the trainer =========
    result = offpolicy_trainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,
        max_epoch=50,
        step_per_epoch=1000,
        step_per_collect=50,
        episode_per_test=10,
        batch_size=64,
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=0.1,
        test_in_train=False,
        reward_metric=reward_metric,
    )

    # return result, policy.policies[agents[1]]
    print(f"\n==========Result==========\n{result}")
    print("\n(the trained policy can be accessed via policy.policies[agents[1]])")

In [ ]:
performance_benchmark(env)

In [ ]:
import Environment
env = Environment.VehicleJobSchedulingEnvACE()
from pettingzoo.test import performance_benchmark
import cProfile
cProfile.run("performance_benchmark(env)")

In [ ]:
%conda install scipy

In [ ]:
import Environment
from supersuit import flatten_v0
from tianshou.env import PettingZooEnv
env = Environment.VehicleJobSchedulingEnvACE()

env = PettingZooEnv(env)

env.action_space.shape, env.observation_space

In [ ]:
from pettingzoo.utils.wrappers import BaseWrapper,BaseParallelWrapper,BaseParallelWraper

In [ ]:
from rich import pretty
pretty.install()

In [ ]:
env.reset()
env.step(1)
env.last()

In [ ]:
env = Environment.VehicleJobSchedulingEnvACE()
sp, _ = env.reset()

In [ ]:
sp

In [ ]:
from environment import Environment

env = Environment.VehicleJobSchedulingEnvACE()
env.reset()
for agent in env.agent_iter(10000):
    env.step(env.action_space(agent).sample())

In [ ]:
env.observation['Machine_0']

In [ ]:
dic = env.parameters.cluster.machines[1].observe(
)

In [ ]:
from gymnasium.spaces.utils import flatten, flatten_space
flatten(env.obs, dic)

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [ ]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

In [ ]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the GPU

In [ ]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

In [ ]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

In [ ]:
from environment import Environment

env = Environment.VehicleJobSchedulingEnvACE()
env.reset()


In [ ]:
env.parameters.cluster.machines[1].observe()

In [ ]:
import cProfile
from pettingzoo.test import performance_benchmark
from environment import Environment
env = Environment.VehicleJobSchedulingEnvACE()
cProfile.run("performance_benchmark(env)")


In [ ]:
import cProfile
from tkinter import E
from pettingzoo.test import performance_benchmark, api_test
from environment import Environment

para = Environment.VehicleJobSchedulingParameters(average_per_slot=50)
env = Environment.VehicleJobSchedulingEnvACE(parameter=para)
env.reset()
for agent in env.agent_iter(10000):
    env.step(env.action_space(agent).sample())

In [ ]:
for agent in env.agent_iter(10):
    action = 1
    env.step(action)
    print(action)

In [ ]:
print("env job finished rate", env.finished_job/env.total_job)

In [ ]:
# 调试Allocation Mechanism

In [ ]:
mac = env.parameters.cluster.machines[1]
for job in env.get_job_next_step():
    job1 = job
    break
mac.show(),job1.show()

In [ ]:
mac.allocate_job(job1)
mac.show(),job1.show()

In [ ]:
import numpy as np

list = [1,5,3,8,2,10,4,9,7,6]
%timeit np.partition(list, 1)[1]

In [ ]:
%timeit np.sort(list)[1]

In [ ]:
from environment import Environment, AllocationMechanism, Machine

env = Environment.VehicleJobSchedulingEnvACE()
for job in env.get_job_next_step():
    job1 = job
    break
cluster = env.parameters.cluster

fp = AllocationMechanism.FirstPrice()
bids = Machine.Bids(cluster,job1)

In [ ]:
bids.request_bids()
bids.get_bids()
bids.bids

In [4]:
from tianshou.data import Collector
from tianshou.env import DummyVectorEnv
from tianshou.policy import MultiAgentPolicyManager, RandomPolicy

import tianshou_setup

env = tianshou_setup.get_env()
policies = MultiAgentPolicyManager(
    [
        RandomPolicy(
            observation_space=env.observation_space, action_space=env.action_space
        )
        for _ in range(len(env.agents))
    ],
    env,
)

2023-09-25 16:45:19.746325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 16:45:20.925905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
pol = RandomPolicy(
            observation_space=env.observation_space, action_space=env.action_space
        )

In [8]:
pol.forward()

TypeError: forward() missing 1 required positional argument: 'batch'